<a href="https://colab.research.google.com/github/TK-Problem/Python-mokymai/blob/master/ML_ir_AI/AI_portreto_generatorius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dreambooth AI sugeneruoto portreto projektas

Šis projektas buvo adaptuotas pagal buildspace [AI Avatar project](https://buildspace.so/builds/ai-avatar) . Rekomenduoju užsiregistruoti ir savarankiškai pereiti per šį projektą. Tai padaryti galite nukeliavę į šią [nuorodą](https://buildspace.so/p/build-ai-avatars).

Builspace sugeneravo šį google colab norebook'ą iš kito projekto [Dreambooth diffusers by Shivam Shrirao](https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth).  Apache-2.0 license observed.

# I. žingsnis - Prisijungti prie virtulios mašinos

Šis projektas skirtas leisti ant google nemokamai prieinamų grafinių kortų (GPU). Pasitikrinkite per `Edit` -> `Notebook settings`, kad yra pasirinkta GPU.


![image info](https://media.springernature.com/lw685/springer-static/image/chp%3A10.1007%2F978-1-4842-4470-8_7/MediaObjects/463852_1_En_7_Fig4_HTML.jpg)



In [ ]:
#@markdown Ši kodo eilutė yra skirta paordyti, kokią grafinę kortą ir kiek laisvų RAM skyrė google
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

# II. žingsnis - Įrašyti bibliotekas
Žemiau esantis kodo blokas yra skirtas įrašyti Python bibliotekoms į virtualią aplinką.

In [ ]:
!wget -q https://raw.githubusercontent.com/buildspace/diffusers/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate==0.15.0 transformers ftfy bitsandbytes==0.35.0 gradio natsort

# III. žingsnis - Prisijungti prie HuggingFace 🤗

Reikia susikurti paskyrą HuggingFace, kad galima būtų iš atsisiųsti `Stable Diffusion` paruoštą modelį. Nueikit į [HuggingFace](huggingface.co?ref=buildspace) ir susikurkite nemokamą paskyrą.

Kai susikurtsite paskyrą, reikia susigeneruoti `read` žetoną [tokens](https://huggingface.co/settings/tokens) puslapį.

![token create](https://hackmd.io/_uploads/BJ1wdTNqi.png)

Šiam projekte naudojame `Stable Diffusion v1.5` sukurtą `Runway`, jo licenziją galite peržiūrėti čia [modelio aprašymas](https://huggingface.co/runwayml/stable-diffusion-v1-5).

In [ ]:
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = "hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

# IV. žingsnis - įsirašyti xformers sukompiliuota biblioteka
xformers yra papildoma biblioteka, kurios reikia dirbant su kalbos interpretavimu ir teksto clasifikavimu.

In [ ]:
%pip install -q https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl
# šis kodas buvo sukompiliuotas ant Tesla T4.

# jei Jums kodas neveikia, tuomet nuimkite komentarus nuo žemiau esančios eilutės
# (CTRL + /) ir įrašykite nekompiliuotą modelį modelį (gali užtrukti kokias 40 min)
# %pip install git+https://github.com/facebookresearch/xformers@4c06c79#egg=xformers

# V. žingsnis - sukonfiguruoti modelį
As mentioned, we're going to be using HuggingFace to load the Stable Diffusion model. There's lots of differently tuned SD models on HF, we're going to stick with the standard v1.5 released by Runway.

The way you choose is a model is by putting in the path of the URL on HuggingFace. So `https://huggingface.co/runwayml/stable-diffusion-v1-5` becomes `runwayml/stable-diffusion-v1-5`. 

You're welcome to try other versions, but we've only tested this on v1.5 and v2.1!

In [ ]:
#@markdown Check this if you want to save the weights in your Google Drive  (takes around 4-5 GB).
#@markdown We won't need to be doing this. 

save_to_gdrive = False #@param {type:"boolean"}

if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

#@markdown Name/path of the initial model.
MODEL_NAME = "runwayml/stable-diffusion-v1-5" #@param {type:"string"}

#@markdown Enter the Gdrive directory to save model at.

OUTPUT_DIR = "stable_diffusion_weights/test-version" #@param {type:"string"}
if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/content/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR

# VI. žingsnis - sukonfiguruoti resursus modelio pertreniravimui
We won't need to touch any of this, but it's here if you want to come back and try turning the knobs once you understand this stuff!

Use the table below to choose the best flags based on your memory and speed requirements. Tested on Tesla T4 GPU.


| `fp16` | `train_batch_size` | `gradient_accumulation_steps` | `gradient_checkpointing` | `use_8bit_adam` | GB VRAM usage | Speed (it/s) |
| ---- | ------------------ | ----------------------------- | ----------------------- | --------------- | ---------- | ------------ |
| fp16 | 1                  | 1                             | TRUE                    | TRUE            | 9.92       | 0.93         |
| no   | 1                  | 1                             | TRUE                    | TRUE            | 10.08      | 0.42         |
| fp16 | 2                  | 1                             | TRUE                    | TRUE            | 10.4       | 0.66         |
| fp16 | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 1.14         |
| no   | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 0.49         |
| fp16 | 1                  | 2                             | TRUE                    | TRUE            | 11.56      | 1            |
| fp16 | 2                  | 1                             | FALSE                   | TRUE            | 13.67      | 0.82         |
| fp16 | 1                  | 2                             | FALSE                   | TRUE            | 13.7       | 0.83          |
| fp16 | 1                  | 1                             | TRUE                    | FALSE           | 15.79      | 0.77         |


Add `--gradient_checkpointing` flag for around 9.92 GB VRAM usage.

remove `--use_8bit_adam` flag for full precision. Requires 15.79 GB with `--gradient_checkpointing` else 17.8 GB.

remove `--train_text_encoder` flag to reduce memory usage further, degrades output quality.

## Bonus - informacija ant kurios pertreniruosi savo modelį
Here's where you tell Stable diffusion *what* you're tuning for. 

**Instance prompt**: this describes exactly what your images are of. In our case it's whatever we decided as the name ("abraza" for me) and "man/woman/person". This is the **label** for the images we uploaded.

**Class prompt**: this just describes what else Stable Diffusion should relate your model to. "man", "woman" or "person" works :)

All you need to do is put your unique identifier ("abraza") here and whatever the class is right here. Make sure you run both blocks!

In [ ]:
INSTANCE_NAME = 'zdvn' #@param {type:"string"}
CLASS_NAME = 'woman' #@param {type:"string"}

In [ ]:
# You can also add multiple concepts here. Try tweaking `--max_train_steps` accordingly.

concepts_list = [
    {
        "instance_prompt":      f"photo of {INSTANCE_NAME} {CLASS_NAME}",
        "class_prompt":         f"photo of a {CLASS_NAME}",
        "instance_data_dir":    f"/content/data/{INSTANCE_NAME}",
        "class_data_dir":       f"/content/data/{CLASS_NAME}"
    }
]

# `class_data_dir` contains regularization images
import json
import os
for c in concepts_list:
    os.makedirs(c["instance_data_dir"], exist_ok=True)

with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)

## Bonus - paveiksliukų įkėlimas

In [ ]:
#@markdown Upload your images by running this cell (recommended). 

#@markdown Run this block and the "choose files" button will pop up. Remember - no more than 10 pictures!

#@markdown OR

#@markdown You can use the file manager on the left panel to upload (drag and drop) to each `instance_data_dir` (it uploads faster)

import os
from google.colab import files
import shutil

for c in concepts_list:
    print(f"Uploading instance images for `{c['instance_prompt']}`")
    uploaded = files.upload()
    for filename in uploaded.keys():
        dst_path = os.path.join(c['instance_data_dir'], filename)
        shutil.move(filename, dst_path)

# VII. žingsnis -treniruoti modelį!
Okay this may seem intimidating, but you don't have to touch most of it!

Again, I've left these in here if you really know what you're doing and want to customise your model, for your first time all you need to do is:
1. Change `max_train_steps`. You wanna keep this number lower than 2000 - the higher it goes, the longer training takes and the more "familiar" SD becomes with you. Keep this number small to avoid overfitting. The general rule of thumb here is 100 steps for each picture+ a base of100. So for 6 pictures, just set it to 700! 
2. **Update `save_sample_prompt` to a prompt with your subject.** Right after training, this block will generate 4 images of you with this prompt. I recommend spazzing it up a bit more than just "Photo of xyz person", those come out quite boring. Put those prompting skills to use!

### This will take ~20m to run!

In [ ]:
!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=50 \
  --sample_batch_size=4 \
  --max_train_steps=500 \
  --save_interval=10000 \
  --save_sample_prompt="portrait of zdvn" \
  --concepts_list="concepts_list.json"

# Reduce the `--save_interval` to lower than `--max_train_steps` to save weights from intermediate steps.
# `--save_sample_prompt` can be same as `--instance_prompt` to generate intermediate samples (saved along with weights in samples directory).

## Step 7.3 - Generate test images!

In [ ]:
#@markdown Specify the weights directory to use (leave blank for latest)
WEIGHTS_DIR = "" #@param {type:"string"}
if WEIGHTS_DIR == "":
    from natsort import natsorted
    from glob import glob
    import os
    WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")

In [ ]:
#@markdown Run to generate a grid of preview images from the last saved weights.
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

weights_folder = OUTPUT_DIR
folders = sorted([f for f in os.listdir(weights_folder) if f != "0"], key=lambda x: int(x))

row = len(folders)
col = len(os.listdir(os.path.join(weights_folder, folders[0], "samples")))
scale = 4
fig, axes = plt.subplots(row, col, figsize=(col*scale, row*scale), gridspec_kw={'hspace': 0, 'wspace': 0})

for i, folder in enumerate(folders):
    folder_path = os.path.join(weights_folder, folder)
    image_folder = os.path.join(folder_path, "samples")
    images = [f for f in os.listdir(image_folder)]
    for j, image in enumerate(images):
        if row == 1:
            currAxes = axes[j]
        else:
            currAxes = axes[i, j]
        if i == 0:
            currAxes.set_title(f"Image {j}")
        if j == 0:
            currAxes.text(-0.1, 0.5, folder, rotation=0, va='center', ha='center', transform=currAxes.transAxes)
        image_path = os.path.join(image_folder, image)
        img = mpimg.imread(image_path)
        currAxes.imshow(img, cmap='gray')
        currAxes.axis('off')
        
plt.tight_layout()
plt.savefig('grid.png', dpi=72)

# Step 8 - Convert weights to CKPT
Since we want to use our fancy new tuned SD with web apps, we'll have to convert it to CKPT. 

If you want to save the CKPT file to your GDrive for the future and are running out of space, you can convert it to fp16, which halves the size but also severely degrades the quality. I recommend **leaving it unchecked** cause we're just going to upload it to HuggingFace at the end. 

In [ ]:
#@markdown Run this block to start the conversion (necessary)
ckpt_path = WEIGHTS_DIR + "/model.ckpt"
#@markdown ----
#@markdown  Check this box to convert to fp16, takes half the space (2GB). Not necessary and not recommmended.
half_arg = ""
fp16 = True #@param {type: "boolean"}

if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path $WEIGHTS_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

## Inference

In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

model_path = WEIGHTS_DIR             # If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe = StableDiffusionPipeline.from_pretrained(model_path, scheduler=scheduler, safety_checker=None, torch_dtype=torch.float16).to("cuda")

g_cuda = None

In [ ]:
#@markdown Can set random seed here for reproducibility.
g_cuda = torch.Generator(device='cuda')
seed = 52362 #@param {type:"number"}
g_cuda.manual_seed(seed)

In [ ]:
#@markdown Run Gradio UI for generating images.
import gradio as gr

def inference(prompt, negative_prompt, num_samples, height=512, width=512, num_inference_steps=50, guidance_scale=7.5):
    with torch.autocast("cuda"), torch.inference_mode():
        return pipe(
                prompt, height=int(height), width=int(width),
                negative_prompt=negative_prompt,
                num_images_per_prompt=int(num_samples),
                num_inference_steps=int(num_inference_steps), guidance_scale=guidance_scale,
                generator=g_cuda
            ).images

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", value="photo of zwx dog in a bucket")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="")
            run = gr.Button(value="Generate")
            with gr.Row():
                num_samples = gr.Number(label="Number of Samples", value=4)
                guidance_scale = gr.Number(label="Guidance Scale", value=7.5)
            with gr.Row():
                height = gr.Number(label="Height", value=512)
                width = gr.Number(label="Width", value=512)
            num_inference_steps = gr.Slider(label="Steps", value=50)
        with gr.Column():
            gallery = gr.Gallery()

    run.click(inference, inputs=[prompt, negative_prompt, num_samples, height, width, num_inference_steps, guidance_scale], outputs=gallery)

demo.launch(debug=True)